### Install and import required packages

In [19]:
!pip install pyLDAvis==3.4.0

   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.6 MB 4.2 MB/s eta 0:00:01
   -------------------- ------------------- 1.3/2.6 MB 3.7 MB/s eta 0:00:01
   -------------------------------- ------- 2.1/2.6 MB 3.7 MB/s eta 0:00:01
   -------------------------------- ------- 2.1/2.6 MB 3.7 MB/s eta 0:00:01
   -------------------------------- ------- 2.1/2.6 MB 3.7 MB/s eta 0:00:01
   ------------------------------------ --- 2.4/2.6 MB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 2.6/2.6 MB 1.8 MB/s eta 0:00:00


In [23]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.preprocessing import LabelEncoder
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pyLDAvis
import pyLDAvis.sklearn
import warnings
warnings.filterwarnings("ignore")

nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\CHELSA\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\CHELSA\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Load and Preprocess the Data

In [27]:
# Load 20 Newsgroups dataset
newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
documents = newsgroups.data

def preprocess(text):
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))
    
    tokens = nltk.word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalpha() and word not in stop_words]
    return ' '.join(tokens)

processed_docs = [preprocess(doc) for doc in documents[:2000]] 

### Vectorization (Count Vectorizer)

In [28]:
vectorizer = CountVectorizer(max_df=0.9, min_df=10, stop_words='english')
X = vectorizer.fit_transform(processed_docs)

### Fit LDA Model

In [29]:
lda = LatentDirichletAllocation(n_components=10, random_state=42)
lda.fit(X)

LatentDirichletAllocation(random_state=42)

### Show Top Keywords per Topic

In [45]:
feature_names = vectorizer.get_feature_names_out()
for idx, topic in enumerate(lda.components_):
    print(f"\nTopic #{idx + 1}:")
    print(" ".join([feature_names[i] for i in topic.argsort()[:-11:-1]]))


Topic #1:
file image use key internet user information data address gif

Topic #2:
like right article bike really van think big shot new

Topic #3:
book history text bible read copy university greek school year

Topic #4:
ed fbi police agent arab batf said israel jewish april

Topic #5:
game year team player know like really good going think

Topic #6:
car time like problem know little way good light thing

Topic #7:
window card use drive work thanks program problem know using

Topic #8:
armenian space year state muslim people new station palestinian russian

Topic #9:
people think know say make right thing way like time

Topic #10:
god law jesus christian point say church human sin christ


### Monkey-patch CountVectorizer to make PyLDAvis work

In [55]:
# Monkey-patch for compatibility
from sklearn.feature_extraction.text import CountVectorizer

if not hasattr(CountVectorizer, 'get_feature_names'):
    CountVectorizer.get_feature_names = CountVectorizer.get_feature_names_out


### Visualize Topics with pyLDAvis

In [57]:
import pyLDAvis
import pyLDAvis.sklearn
import warnings
warnings.filterwarnings("ignore")

pyLDAvis.enable_notebook()

# Prepare the visualization panel
panel = pyLDAvis.sklearn.prepare(
    lda_model=lda, 
    dtm=X, 
    vectorizer=vectorizer
)
panel


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8      0.017247 -0.085810       1        1  19.746847
6     -0.163645  0.146950       2        1  16.940950
0     -0.044304  0.193372       3        1  12.976843
7      0.177354  0.065248       4        1   9.377128
5     -0.097913 -0.012348       5        1   9.206811
9      0.013585 -0.170020       6        1   6.844396
4     -0.126753 -0.066115       7        1   6.733462
2      0.084110 -0.059835       8        1   6.226481
3      0.215349  0.066870       9        1   6.040199
1     -0.075030 -0.078311      10        1   5.906883, topic_info=        Term        Freq       Total Category  logprob  loglift
788      god  317.000000  317.000000  Default  30.0000  30.0000
705     file  404.000000  404.000000  Default  29.0000  29.0000
574       ed  162.000000  162.000000  Default  28.0000  28.0000
765     game  179.000000  179.000000  Default  27.0000  27.0000
896    image  264.000000  264.000000  Default  26.0000  26.0000
...      ...         ...         ...      ...      ...      ...
423   course   23.145183  125.048655  Topic10  -5.5069   1.1421
1017    know   28.341813  608.012286  Topic10  -5.3043  -0.2368
1237    need   24.249784  304.283520  Topic10  -5.4603   0.2995
1408    post   23.020072  170.247406  Topic10  -5.5123   0.8282
1942    time   23.736774  588.912932  Topic10  -5.4816  -0.3822

[619 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
5         1  0.253580  accept
5         2  0.108677  accept
5         3  0.036226  accept
5         6  0.470934  accept
5         7  0.018113  accept
...     ...       ...     ...
2136      4  0.700795    york
2136      7  0.092210    york
2136      8  0.036884    york
2136      9  0.036884    york
2136     10  0.129094    york

[2083 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 7, 1, 8, 6, 10, 5, 3, 4, 2])

### Save pyLDAvis --> HTML file

In [59]:
pyLDAvis.save_html(panel, 'lda_visualization.html')

### Testing the LDA Model

In [63]:
sample_texts = [
    "The resurrection of Jesus is a fundamental belief in Christianity.",
    "Intel just released their latest i9 processor with improved thermal performance.",
    "NASA’s new rover has landed on Mars to search for signs of life.",
    "The government’s decision on abortion laws has sparked protests."
]

def preprocess(text):
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))
    tokens = nltk.word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalpha() and word not in stop_words]
    return " ".join(tokens)

processed_samples = [preprocess(text) for text in sample_texts]

X_new = vectorizer.transform(processed_samples)

topic_probs = lda.transform(X_new)

feature_names = vectorizer.get_feature_names_out()

for i, probs in enumerate(topic_probs):
    top_topic = probs.argmax()
    print(f"\nDocument {i+1}: {sample_texts[i]}")
    print(f"→ Most likely Topic #{top_topic} with probability {probs[top_topic]:.4f}")
    print(f"Top keywords in this topic:")
    print(" ".join([feature_names[i] for i in lda.components_[top_topic].argsort()[:-11:-1]]))


Document 1: The resurrection of Jesus is a fundamental belief in Christianity.
→ Most likely Topic #9 with probability 0.5004
Top keywords in this topic:
god law jesus christian point say church human sin christ

Document 2: Intel just released their latest i9 processor with improved thermal performance.
→ Most likely Topic #6 with probability 0.6833
Top keywords in this topic:
window card use drive work thanks program problem know using

Document 3: NASA’s new rover has landed on Mars to search for signs of life.
→ Most likely Topic #7 with probability 0.6382
Top keywords in this topic:
armenian space year state muslim people new station palestinian russian

Document 4: The government’s decision on abortion laws has sparked protests.
→ Most likely Topic #8 with probability 0.6097
Top keywords in this topic:
people think know say make right thing way like time
